# Chapter 35: ABCs and the Copy Protocol

This notebook explores Abstract Base Classes (ABCs) for defining interfaces, virtual subclass registration, and Python's copy and pickle protocols for controlling how objects are duplicated and serialized.

## Key Concepts
- **`ABC` and `@abstractmethod`**: Define interfaces that enforce method implementation
- **Virtual subclasses**: Register classes as subclasses without inheritance via `ABC.register`
- **`__subclasshook__`**: Customize `isinstance` and `issubclass` checks
- **`copy.copy` / `copy.deepcopy`**: Shallow and deep copying of objects
- **`__copy__` / `__deepcopy__`**: Customize copy behavior
- **Pickle protocol**: `__getstate__` / `__setstate__` for serialization control

## Section 1: Abstract Base Classes (ABCs)

An ABC defines an interface by declaring abstract methods. Subclasses must implement all abstract methods before they can be instantiated.

In [ ]:
from abc import ABC, abstractmethod


class Shape(ABC):
    """Abstract base class for geometric shapes."""

    @abstractmethod
    def area(self) -> float:
        """Return the area of the shape."""
        ...

    @abstractmethod
    def perimeter(self) -> float:
        """Return the perimeter of the shape."""
        ...

    def describe(self) -> str:
        """Concrete method available to all subclasses."""
        return f"{type(self).__name__}: area={self.area():.2f}, perimeter={self.perimeter():.2f}"


# Cannot instantiate an ABC directly
try:
    Shape()
except TypeError as e:
    print(f"TypeError: {e}")

In [ ]:
import math


class Square(Shape):
    """Concrete subclass implementing all abstract methods."""

    def __init__(self, side: float) -> None:
        self.side = side

    def area(self) -> float:
        return self.side ** 2

    def perimeter(self) -> float:
        return 4 * self.side


class CircleShape(Shape):
    """Another concrete subclass."""

    def __init__(self, radius: float) -> None:
        self.radius = radius

    def area(self) -> float:
        return math.pi * self.radius ** 2

    def perimeter(self) -> float:
        return 2 * math.pi * self.radius


# Both concrete subclasses can be instantiated
s = Square(4.0)
c = CircleShape(3.0)

print(s.describe())
print(c.describe())

# isinstance checks work
print(f"\nisinstance(s, Shape): {isinstance(s, Shape)}")
print(f"isinstance(c, Shape): {isinstance(c, Shape)}")
print(f"s.area() == 16.0: {s.area() == 16.0}")

## Section 2: Incomplete Implementations

If a subclass does not implement all abstract methods, it is still abstract and cannot be instantiated.

In [ ]:
from abc import ABC, abstractmethod


class Animal(ABC):
    @abstractmethod
    def speak(self) -> str: ...

    @abstractmethod
    def move(self) -> str: ...


# Incomplete: only implements speak, not move
class PartialAnimal(Animal):
    def speak(self) -> str:
        return "..."


try:
    PartialAnimal()
except TypeError as e:
    print(f"TypeError: {e}")


# Complete implementation
class Dog(Animal):
    def speak(self) -> str:
        return "Woof!"

    def move(self) -> str:
        return "runs"


dog = Dog()
print(f"\nDog says: {dog.speak()}, {dog.move()}")

## Section 3: Virtual Subclasses with `register`

ABCs support virtual subclass registration. A class can be registered as a subclass of an ABC without actually inheriting from it. This makes `isinstance` and `issubclass` return `True`, but the class does not appear in the MRO.

In [ ]:
from abc import ABC, abstractmethod


class Drawable(ABC):
    @abstractmethod
    def draw(self) -> str: ...


# A regular class (no inheritance from Drawable)
class Circle:
    def draw(self) -> str:
        return "O"


# Register Circle as a virtual subclass of Drawable
Drawable.register(Circle)

c = Circle()
print(f"isinstance(c, Drawable): {isinstance(c, Drawable)}")
print(f"issubclass(Circle, Drawable): {issubclass(Circle, Drawable)}")

# But Drawable is NOT in Circle's MRO
print(f"\nCircle.__mro__: {Circle.__mro__}")
print(f"Drawable in Circle.__mro__: {Drawable in Circle.__mro__}")

# Virtual subclasses are NOT checked for abstract method implementation
class EmptyVirtual:
    pass

Drawable.register(EmptyVirtual)
ev = EmptyVirtual()  # No error even though draw() is not implemented
print(f"\nisinstance(ev, Drawable): {isinstance(ev, Drawable)}")
print(f"hasattr(ev, 'draw'): {hasattr(ev, 'draw')}")

## Section 4: `__subclasshook__` for Structural Typing

The `__subclasshook__` classmethod allows an ABC to customize `isinstance` and `issubclass` checks. This enables structural (duck) typing -- if a class has the right methods, it is considered a subclass.

In [ ]:
from abc import ABC, abstractmethod


class Renderable(ABC):
    @abstractmethod
    def render(self) -> str: ...

    @classmethod
    def __subclasshook__(cls, C: type) -> bool:
        """Any class with a render() method is considered Renderable."""
        if cls is Renderable:
            if hasattr(C, "render") and callable(getattr(C, "render")):
                return True
        return NotImplemented


# This class has render() but does NOT inherit from Renderable
class HTMLWidget:
    def render(self) -> str:
        return "<div>Widget</div>"


# This class does NOT have render()
class DataStore:
    def save(self) -> None:
        pass


widget = HTMLWidget()
store = DataStore()

print(f"isinstance(widget, Renderable): {isinstance(widget, Renderable)}")
print(f"isinstance(store, Renderable):  {isinstance(store, Renderable)}")

# No registration needed -- structural check is automatic
print(f"\nissubclass(HTMLWidget, Renderable): {issubclass(HTMLWidget, Renderable)}")
print(f"issubclass(DataStore, Renderable):  {issubclass(DataStore, Renderable)}")

## Section 5: Shallow Copy with `copy.copy`

A shallow copy creates a new object but does not recursively copy nested objects. The new object shares references to the same inner objects.

In [ ]:
import copy

# Shallow copy of a nested list
original: list[list[int]] = [[1, 2], [3, 4], [5, 6]]
shallow: list[list[int]] = copy.copy(original)

print(f"original: {original}")
print(f"shallow:  {shallow}")

# The outer list is a new object
print(f"\noriginal is shallow: {original is shallow}")
print(f"original == shallow: {original == shallow}")

# But inner lists are shared (same references)
print(f"\noriginal[0] is shallow[0]: {original[0] is shallow[0]}")

# Modifying a shared inner list affects both
shallow[0].append(99)
print(f"\nAfter shallow[0].append(99):")
print(f"original: {original}")
print(f"shallow:  {shallow}")

## Section 6: Deep Copy with `copy.deepcopy`

A deep copy recursively copies all nested objects, creating a fully independent clone.

In [ ]:
import copy

original: list[list[int]] = [[1, 2], [3, 4]]
deep: list[list[int]] = copy.deepcopy(original)

print(f"original: {original}")
print(f"deep:     {deep}")

# Both the outer and inner lists are different objects
print(f"\noriginal is deep:       {original is deep}")
print(f"original[0] is deep[0]: {original[0] is deep[0]}")

# Modifying the deep copy does NOT affect the original
deep[0].append(99)
print(f"\nAfter deep[0].append(99):")
print(f"original: {original}")
print(f"deep:     {deep}")

## Section 7: Custom `__copy__` and `__deepcopy__`

Classes can customize shallow and deep copy behavior by implementing `__copy__` and `__deepcopy__`.

In [ ]:
import copy


class Config:
    """A configuration object that tracks how many times it was copied."""

    def __init__(self, settings: dict[str, object]) -> None:
        self.settings = settings
        self.copy_count: int = 0

    def __copy__(self) -> "Config":
        """Shallow copy: copies the settings dict but shares nested values."""
        new = Config(self.settings.copy())
        new.copy_count = self.copy_count + 1
        return new

    def __repr__(self) -> str:
        return f"Config(settings={self.settings}, copy_count={self.copy_count})"


c1 = Config({"debug": True, "log_level": "INFO"})
c2 = copy.copy(c1)
c3 = copy.copy(c2)

print(f"c1: {c1}")
print(f"c2: {c2}")
print(f"c3: {c3}")

# Settings are independent (shallow-copied dict)
print(f"\nc1.settings is c2.settings: {c1.settings is c2.settings}")

c2.settings["debug"] = False
print(f"c1.settings['debug']: {c1.settings['debug']}")
print(f"c2.settings['debug']: {c2.settings['debug']}")

In [ ]:
import copy


class TreeNode:
    """A tree node that customizes deep copy."""

    def __init__(self, value: str, children: list["TreeNode"] | None = None) -> None:
        self.value = value
        self.children: list[TreeNode] = children or []

    def __deepcopy__(self, memo: dict) -> "TreeNode":
        """Deep copy that tracks visited nodes via memo dict."""
        # Check memo to handle circular references
        if id(self) in memo:
            return memo[id(self)]

        new_node = TreeNode(self.value)
        memo[id(self)] = new_node
        new_node.children = copy.deepcopy(self.children, memo)
        return new_node

    def __repr__(self) -> str:
        child_vals: list[str] = [c.value for c in self.children]
        return f"TreeNode({self.value!r}, children={child_vals})"


# Build a tree
root = TreeNode("root", [
    TreeNode("child1", [TreeNode("grandchild1")]),
    TreeNode("child2"),
])

# Deep copy the entire tree
root_copy = copy.deepcopy(root)

print(f"Original: {root}")
print(f"Copy:     {root_copy}")

# Completely independent
print(f"\nroot is root_copy: {root is root_copy}")
print(f"root.children[0] is root_copy.children[0]: {root.children[0] is root_copy.children[0]}")

# Modify copy without affecting original
root_copy.children[0].value = "MODIFIED"
print(f"\nOriginal child1: {root.children[0].value}")
print(f"Copy child1:     {root_copy.children[0].value}")

## Section 8: Copy with Class Instances

By default, `copy.copy` and `copy.deepcopy` work with most objects. Understanding the default behavior helps you decide when custom `__copy__` or `__deepcopy__` methods are needed.

In [ ]:
import copy


class Team:
    def __init__(self, name: str, members: list[str]) -> None:
        self.name = name
        self.members = members


original = Team("Engineering", ["Alice", "Bob"])

# Shallow copy: new Team object but shares the members list
shallow = copy.copy(original)
print(f"Shallow copy name: {shallow.name}")
print(f"Members shared: {original.members is shallow.members}")

# Deep copy: new Team object AND new members list
deep = copy.deepcopy(original)
print(f"\nDeep copy name: {deep.name}")
print(f"Members shared: {original.members is deep.members}")

# Verify independence of deep copy
deep.members.append("Charlie")
print(f"\nOriginal members: {original.members}")
print(f"Deep copy members: {deep.members}")

## Section 9: Pickle Protocol -- `__getstate__` and `__setstate__`

The `pickle` module serializes Python objects to bytes. Classes can customize serialization with `__getstate__` (what to save) and `__setstate__` (how to restore).

In [ ]:
import pickle


class Connection:
    """A class with non-serializable state that customizes pickling."""

    def __init__(self, host: str, port: int) -> None:
        self.host = host
        self.port = port
        self._socket = f"<socket to {host}:{port}>"  # Simulate non-serializable
        self.connected: bool = True

    def __getstate__(self) -> dict[str, object]:
        """Return state for pickling (exclude _socket)."""
        state: dict[str, object] = self.__dict__.copy()
        del state["_socket"]  # Don't pickle the socket
        state["connected"] = False  # Mark as disconnected
        return state

    def __setstate__(self, state: dict[str, object]) -> None:
        """Restore state from pickle and reconnect."""
        self.__dict__.update(state)
        # Recreate the socket
        self._socket = f"<socket to {self.host}:{self.port}>"
        self.connected = True

    def __repr__(self) -> str:
        return f"Connection({self.host}:{self.port}, connected={self.connected})"


# Pickle and unpickle
conn = Connection("localhost", 5432)
print(f"Before pickle: {conn}")
print(f"Socket: {conn._socket}")

data: bytes = pickle.dumps(conn)
print(f"\nPickled size: {len(data)} bytes")

restored: Connection = pickle.loads(data)
print(f"\nAfter unpickle: {restored}")
print(f"Socket: {restored._socket}")
print(f"Connected: {restored.connected}")

## Section 10: `__reduce__` for Advanced Pickle Control

For more control over pickling, implement `__reduce__`. It returns a tuple that tells pickle how to reconstruct the object.

In [ ]:
import pickle


class Color:
    """A color class that uses __reduce__ for custom pickling."""

    def __init__(self, r: int, g: int, b: int) -> None:
        self.r = r
        self.g = g
        self.b = b

    def __reduce__(self) -> tuple:
        """Return (callable, args) for reconstruction."""
        return (Color, (self.r, self.g, self.b))

    def __repr__(self) -> str:
        return f"Color({self.r}, {self.g}, {self.b})"

    def hex_code(self) -> str:
        return f"#{self.r:02x}{self.g:02x}{self.b:02x}"


original = Color(255, 128, 0)
print(f"Original: {original} -> {original.hex_code()}")

# Pickle round-trip
data: bytes = pickle.dumps(original)
restored: Color = pickle.loads(data)
print(f"Restored: {restored} -> {restored.hex_code()}")

# Verify equality of attributes
print(f"\nSame r: {original.r == restored.r}")
print(f"Same g: {original.g == restored.g}")
print(f"Same b: {original.b == restored.b}")

## Section 11: Combining ABCs with Copy

A practical pattern: define an ABC with a `clone` method that uses `copy.deepcopy` to create independent copies of any concrete subclass.

In [ ]:
import copy
from abc import ABC, abstractmethod


class Prototype(ABC):
    """ABC that provides a clone method via copy.deepcopy."""

    @abstractmethod
    def describe(self) -> str: ...

    def clone(self) -> "Prototype":
        """Create a deep copy of this object."""
        return copy.deepcopy(self)


class Document(Prototype):
    def __init__(self, title: str, pages: list[str]) -> None:
        self.title = title
        self.pages = pages

    def describe(self) -> str:
        return f"Document({self.title!r}, {len(self.pages)} pages)"


# Create and clone a document
doc = Document("Report", ["Introduction", "Analysis", "Conclusion"])
doc_clone = doc.clone()

print(f"Original: {doc.describe()}")
print(f"Clone:    {doc_clone.describe()}")

# Clone is fully independent
doc_clone.title = "Report v2"
doc_clone.pages.append("Appendix")

print(f"\nAfter modifying clone:")
print(f"Original: {doc.describe()}, pages={doc.pages}")
print(f"Clone:    {doc_clone.describe()}, pages={doc_clone.pages}")

# isinstance checks
print(f"\nisinstance(doc_clone, Prototype): {isinstance(doc_clone, Prototype)}")
print(f"isinstance(doc_clone, Document):  {isinstance(doc_clone, Document)}")

## Summary

### Abstract Base Classes
- **`class MyABC(ABC)`**: Create an abstract base class
- **`@abstractmethod`**: Declare methods that subclasses must implement
- **Instantiation check**: ABCs with unimplemented abstract methods raise `TypeError`
- **Concrete methods**: ABCs can include non-abstract methods that subclasses inherit

### Virtual Subclasses
- **`MyABC.register(SomeClass)`**: Register a class as a virtual subclass
- **`isinstance` / `issubclass`**: Return `True` for registered classes
- **No MRO change**: Virtual subclasses do not inherit from the ABC
- **`__subclasshook__`**: Customize structural type checking (duck typing)

### Copy Protocol
- **`copy.copy(obj)`**: Shallow copy -- new object, shared nested references
- **`copy.deepcopy(obj)`**: Deep copy -- recursively copies all nested objects
- **`__copy__(self)`**: Customize shallow copy behavior
- **`__deepcopy__(self, memo)`**: Customize deep copy with circular reference tracking

### Pickle Protocol
- **`__getstate__`**: Return the state to be serialized (exclude non-serializable data)
- **`__setstate__`**: Restore object from deserialized state
- **`__reduce__`**: Return `(callable, args)` for advanced reconstruction control
- Use pickle for persistence; use copy for in-memory duplication